# **Assignment 3: LMMs**

For this assignment, you will be using colab.

Please complete and submit this assignment by February 21, 11:59 PM. Download and
submit the .ipynb file and share the notebook with the TA (swetha.sirnam@ucf.edu and
swethacrcv@gmail.com)

## Useful Resources:

**PyTorch Colab Documentation:**
https://pytorch.org/tutorials/beginner/colab.html

**HuggingFace Sample Notebooks:**
https://huggingface.co/docs/transformers/en/notebooks

**LlaVA-OneVision**
Model Doc:
https://huggingface.co/docs/transformers/en/model_doc/llava_onevision
Weights: https://huggingface.co/lmms-lab/llava-onevision-qwen2-0.5b-ov

**PaliGemma**
Model Doc: https://huggingface.co/docs/transformers/en/model_doc/paligemma
Weights: https://huggingface.co/google/paligemma-3b-mix-224

**Qwen2VL**
Model Doc: https://huggingface.co/docs/transformers/en/model_doc/qwen2_vl
Weights: https://huggingface.co/Qwen/Qwen2-VL-2B-Instruct\
Full Family: https://huggingface.co/collections/Qwen/qwen2-vl-
66cee7455501d7126940800d


Sample Image: http://images.cocodataset.org/val2017/000000039769.jpg


**Platinum Bench**
Dataset: https://huggingface.co/datasets/madrylab/platinum-bench
Paper: https://arxiv.org/abs/2502.03461


## Tasks:

1. Setup a Google Colab and load the above 3 models onto GPU from huggingface
and show memory usage for each model. **[10 points]**

2. Run inference on all 3 models on the sample image above to generate detailed
description. **[10 points]**

3. Evaluate each model successively on the clean (consensus + verified +
revised ) part of platinum bench’s vqa and gsm8k subsets [3 x 10 + 3 x 10 = 60
points]

4. Scaling experiments: Evaluate on vqa and gsm8k subsets (as in Task 3) on
Qwen2VL and analyze the impact of scaling, compare the performance and
inference time. [2 x 10 = 20 points]


# Import Packages

In [ ]:
# importing os module for environment variables
import os
# importing necessary functions from dotenv library
from dotenv import load_dotenv, dotenv_values 
# loading variables from .env file
load_dotenv() 

# Install HuggingFace in system if not installed
%pip install python-dotenv
%pip install git+https://github.com/huggingface/transformers
%pip install --upgrade huggingface_hub
%pip install --upgrade diffusers transformers accelerate mediapy peft pytorch_fid
%pip install torch torchvision torchaudio

# Set model cache location 
import os
os.environ['HF_HOME'] = os.getenv("CACHE_LOCATION")
!export HF_HOME={os.getenv("CACHE_LOCATION")}


# Login to HuggingFace
# This code will save huggingface token to PC, but your PC first has to have github token stored in pc
!huggingface-cli login --token {os.getenv("HUGGINGFACE_TOKEN")} --add-to-git-credential




Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-i7qhyu_d
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-i7qhyu_d
  Resolved https://github.com/huggingface/transformers to commit 9f51dc25357bcde280a02b59e80b66248b018ca4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Token is valid (permission: fineGrained).
The token `Desktop_PC_Ubuntu` has been saved to /mnt/Creative/SoftwareDevelopment/huggingface/cache/stored_tokens
Your token has been saved in your configured git credential h

In [ ]:
# Prints out the location that the models are downloaded to
print(os.getenv("CACHE_LOCATION"))

/mnt/Creative/SoftwareDevelopment/huggingface/cache/


## Import LlaVA-OneVision Model and Check GPU Usage


In [ ]:
from transformers import AutoProcessor, LlavaOnevisionForConditionalGeneration
import torch

llava_model_id = "llava-hf/llava-onevision-qwen2-7b-ov-hf"
llava_processor = AutoProcessor.from_pretrained(llava_model_id) 
llava_model = LlavaOnevisionForConditionalGeneration.from_pretrained(
    llava_model_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto"
)

/home/malneyugfl/anaconda3/envs/huggingface/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 4/4 [00:26<00:00,  6.65s/it]


In [5]:
# Check GPU Memory Usage
!nvidia-smi

Wed Feb 19 01:38:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.120                Driver Version: 550.120        CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090        Off |   00000000:01:00.0  On |                  N/A |
| 30%   32C    P8             41W /  350W |    8118MiB /  24576MiB |     20%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Import PaliGemma Model and Check GPU Usage

In [ ]:
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration

paligemma_model_id = "google/paligemma-3b-mix-224"
paligemma_processor = AutoProcessor.from_pretrained(paligemma_model_id)
paligemma_model = PaliGemmaForConditionalGeneration.from_pretrained(paligemma_model_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto")


Loading checkpoint shards: 100%|██████████| 3/3 [00:19<00:00,  6.66s/it]


In [9]:
# Check GPU Memory Usage
!nvidia-smi

Wed Feb 19 01:50:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.120                Driver Version: 550.120        CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090        Off |   00000000:01:00.0  On |                  N/A |
| 30%   33C    P8             42W /  350W |   11446MiB /  24576MiB |     31%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Import Qwen2VL Model and Check GPU Usage

In [ ]:
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor

qwen2vl_model_id= "Qwen/Qwen2-VL-7B-Instruct"
processor = AutoProcessor.from_pretrained(qwen2vl_model_id)
qwen2vl_model = Qwen2VLForConditionalGeneration.from_pretrained(
    qwen2vl_model_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto" 
    )


Loading checkpoint shards: 100%|██████████| 5/5 [00:40<00:00,  8.10s/it]


In [11]:
# Check GPU Memory Usage
!nvidia-smi

Wed Feb 19 02:14:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.120                Driver Version: 550.120        CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090        Off |   00000000:01:00.0  On |                  N/A |
| 30%   31C    P8             40W /  350W |   11562MiB /  24576MiB |      4%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Run Inference using each model on the Sample Image

## Evaluate each Model on the clean platinum bench's vqa and gsm8k subsets

## Evaluate on vqa and gsm8k subsets (as in Task 3) on Qwen2VL]